# CBurnFS

    a = CBurnFS(bootpath)
    
CBurnFS creates a wrapper around an instance of `MulticelFS` and manages its configuration.

### Management Role

CBurnFS adds and removes members from the `MulticelFS` and configures authentication. It reads and writes state in the `@` provided by the multicel.

### Delegation

The `MulticelFS` is responsible for combining multiple domains and for reporting item membership in the `getinfo()` output.
    
## Init

For each entry in `bootpath`/`.cosm/etc/fstab`, create a Dcel.

    -> dcel_list

Then create a `Dcel(dcel_list, MulticelFS)`

    -> root

Set `root` as the CBurnFS instance's root resource.

    self.root = root

Rock and roll.

In [8]:
from fs.osfs import OSFS
from fs.base import FS
from fs.info import Info
from fs.errors import ResourceNotFound
from fs.copy import copy_dir, copy_file

from MulticelFS import MulticelFS
# don't need? from blackstrap import BlackstrapFS
from Dcel import Dcel # factor out
from APath import APath
from Fudge import Fudge

from copy import deepcopy
import json
import six
from urllib.parse import urlparse
from os.path import split
if six.PY2:
    from urllib import unquote
else:
    from urllib.parse import unquote

#for merge lib (tbd): mergeinfo


_version='0.16'

# Refactoring to simplify dependencies.
# Goal is an independent, installable python module.

# usage:
# cbfs = CBurnFS(bootpath)

def init_dcel_from_url(boot,url):
    dc = None
    return dc

FSTAB_RELPATH = '@/etc/fstab'
FSTAB_ABSPATH = '/@/etc/fstab'

class CBurnFS(APath):        
    
    def __loadFstab(self, bootpath: str):
        boot = Fudge(bootpath)
        # FIXME: must force load of root - consider this a bug
        boot_root = boot/'/'
        fstab = boot/FSTAB_RELPATH
        cels = []
        for ea in fstab:
            if ea/'vfstype' == 'cburnfs':
                urlstr = ea/'spec'
                _scheme = ea/'spec.url/scheme'
                if _scheme == '':
                    _scheme = 'file'
                service_class = boot._apath.cosm['services'][str(_scheme)].value
                try:
                    cels += [ Dcel(address=str(urlstr), 
                                service_class=service_class) ]
                except:
                    pass
        root = Dcel(
            service_class=MulticelFS,
            address=cels
        )
        return root
                
    def __init__(self, bootpath: str):
        self._bootpath = bootpath
        root = self.__loadFstab(bootpath)
        super().__init__(root)
        
    def _reinit(self):
        print(f"CBurnFS::_reinit() called.")
        root = self.__loadFstab(self._bootpath)
        super()._reinit(root)
        
        
    # updater junk
    def updateHosts(self,path,hosts):
        
        dcel = self.target
        svc = dcel.service
        
        _path = unquote(path)
        pathDir,pathBase = split(_path)
        
        if type(svc) == MulticelFS:   
            for host in hosts:
                
                # get host fs
                dest = svc.get_dcel_by_host(host)
            
                # make path on host fs
                destSvc = dest.service
                destDirFS = destSvc.makedirs(pathDir,None,True)
                
                # copy path from multifs
                #   to path in host fs
                
                if svc.getinfo(_path).is_dir:
                    copy_dir(svc,_path,destDirFS,pathBase)
                else:
                    copy_file(svc,_path,destDirFS,pathBase)            
            

    def removeHosts(self,path,hosts):
        
        _path = unquote(path)
        pathDir,pathBase = split(_path)
        dcel = self.target
        svc = dcel.service
        
        if type(svc) == MulticelFS:   
            for host in hosts:
                # get host fs
                dest = svc.get_dcel_by_host(host)
        
                if dest.getinfo(_path).is_dir:
                    dest.service.removetree(_path)
                else:
                    dest.service.remove(_path)
                    
    def urlListFromDict(self, path, ob) -> list:
        dirlist = list()
        if type(ob) is dict:
            for each in ob:
                dirlist = dirlist + self.urlListFromDict(f"{path}/{each}", ob[each])
        else:
            return [(path, ob)]
        return dirlist

    def updateMultiValue(self, path, multiValue):
        """multiValue should already have been parsed."""
        _path = unquote(path)
        target = self.target
        svc = target.service
        actionPairs = self.urlListFromDict(_path, multiValue)
        root_fu = Fudge(self)
        for url,val in actionPairs:
            (root_fu/url)['.'] = val
    
    # webdav style updaters
    
    def propertyupdate(self,path,uprq):
        
        print(f"CBurnFS::propertyupdate(): path={path}, uprq={uprq}")
        
        for verb in uprq:
            if verb == 'append':
                if "cburn" in uprq[verb]:
                    target = uprq[verb]["cburn"]
                    if "hosts" in target:
                        self.updateHosts(
                            path,
                            target["hosts"])
                    if "multivalue" in target:
                        self.updateMultiValue(
                            path,
                            target["multivalue"])
                        if(path.startswith(FSTAB_RELPATH)
                           or path.startswith(FSTAB_ABSPATH)):
                            self._reinit()
            
            if verb == 'remove':
                if "cburn" in uprq[verb]:
                    target = uprq[verb]["cburn"]
                    if "hosts" in target:
                        self.removeHosts(path,
                            target["hosts"])
                
        return 'CBurnFS.propertyupdate():return: improve this response'
    
    def processRequest(self,path,rq):
        # rq is a dict object from flask app.
        print(f"CBurnFS::processRequest() rq={rq}")
        response = dict()
        for mod in rq:
            if mod == 'propertyupdate':
                response[mod] = self.propertyupdate(path,rq[mod])
        return response
    

## Todo

- cburnfs: config location
- cburnfs: check for presence of '@/etc/fstab' and error informatively
- demodata: location, generator
- blackstrap: yaml config
- hiena: yaml grammar definitions

### Make it Just Work

    cbfs = CBurnFS('.')
    
Should initialize a `CBurnFS` rooted at the input path.

Lookup of @/etc/fstab should fail silently
-- or at least provide an educational message that 
the file system is running on a single layer.

Adding/Modifying an @/etc/fstab should trigger a
re-build of the layers.

Adding/Modifying other aspects of @ should modify
the Apath Cosm --- provide for extending the services
for URL's -- such as s3, file, sftp etc.


### SETUP TEST ENVIRONMENT

In [9]:
# This needs to be tweaked for your environment.
# It references mount locations that may not exist.
import demo_blackstrap_config

In [10]:
# TODO: need to check for presence of '@/etc/fstab' and fail gracefully
cbfs = CBurnFS('file://fs.localhost')

In [11]:
cbfs.writetext('/test.txt','Hola, Amigo!')

In [12]:
cbfs.readtext('/test.txt')

'Hola, Amigo!'

### TEST removeHosts via propertyupdate()

In [5]:
import json

In [6]:
path = '/fruit/apples.txt'
cbfs.getinfo(path).raw

{'basic': {'name': 'apples.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost']}

In [6]:
updateJSON = '{"propertyupdate":{"remove":{"cburn":{"hosts":["file://fs4.localhost"]}}}}'
updateRequest = json.loads(updateJSON)

In [8]:
print(cbfs.getinfo(path).raw)
cbfs.processRequest(path,updateRequest)
print(cbfs.getinfo(path).raw)

{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost']}


### TEST updateHosts via propertyupdate()

In [10]:
import json

In [11]:
path = 'fruit/apples.txt'
cbfs.getinfo(path).raw

{'basic': {'name': 'apples.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost']}

In [12]:
updateJSON = '{"propertyupdate":{"append":{"cburn":{"hosts":["file://fs4.localhost"]}}}}'
updateRequest = json.loads(updateJSON)

In [13]:
print(cbfs.getinfo(path).raw)
cbfs.processRequest(path,updateRequest)
print(cbfs.getinfo(path).raw)

{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost']}
CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'hosts': ['file://fs4.localhost']}}}}
CBurnFS::propertyupdate(): path=fruit/apples.txt, uprq={'append': {'cburn': {'hosts': ['file://fs4.localhost']}}}
{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost', 'file://fs4.localhost']}


### TEST _reinit() HOOK ON propertyupdate()

In [26]:
import json

In [27]:
# propertyupdate JSON
updateJSON = '{"propertyupdate":{"append":{"cburn":{"multivalue":{"1":{"spec":"file://fs4.localhost"},"2":{"mntopts.cskvp":{"shortid":"FS1"}},"3":{"mntopts.cskvp":{"shortid":"FS3"}}}}}}}'
updateRequest = json.loads(updateJSON)
path = "@/etc/fstab.fstab"
cbf = Fudge(cbfs)
fstab = cbf/path

In [28]:
print(fstab/'1/spec')

file://fs4.localhost


In [29]:
cbfs.processRequest(path,updateRequest)

CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'spec': 'file://fs4.localhost'}, '2': {'mntopts.cskvp': {'shortid': 'FS1'}}, '3': {'mntopts.cskvp': {'shortid': 'FS3'}}}}}}}
CBurnFS::propertyupdate(): path=@/etc/fstab.fstab, uprq={'append': {'cburn': {'multivalue': {'1': {'spec': 'file://fs4.localhost'}, '2': {'mntopts.cskvp': {'shortid': 'FS1'}}, '3': {'mntopts.cskvp': {'shortid': 'FS3'}}}}}}
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
CBurnFS::_reinit() called.


{'propertyupdate': 'CBurnFS.propertyupdate():return: improve this response'}

### TEST MulticelFS via getinfo()

In [12]:
i = cbfs.getinfo('/fruit/cherries.html.txt')

In [13]:
print(i.raw)

{'basic': {'name': 'cherries.html.txt', 'is_dir': False}, 'hosts': ['file://fs3.localhost']}


### TEST CBurnFS::propertyupdate() and updateMultiValue()

In [15]:
import json

In [16]:
# propertyupdate JSON
updateJSON = '{"propertyupdate":{"append":{"cburn":{"multivalue":{"1":{"mntopts.cskvp":{"shortid":"fs2"}},"2":{"mntopts.cskvp":{"shortid":"fs1"}},"3":{"mntopts.cskvp":{"shortid":"fs3"}}}}}}}'
updateRequest = json.loads(updateJSON)
print(updateRequest)

{'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}}


In [15]:
# locate path relative to the above request
path = "@/etc/fstab.fstab"
cbf = Fudge(cbfs)
fstab = cbf/path
print(fstab)

{'1': {'spec': <Dcel.Dcel object at 0x7fa9df966fa0>, 'file': <Dcel.Dcel object at 0x7fa9df966580>, 'vfstype': <Dcel.Dcel object at 0x7fa9df966a90>, 'mntopts': <Dcel.Dcel object at 0x7fa9df966ca0>, 'freq': <Dcel.Dcel object at 0x7fa9df966970>, 'passno': <Dcel.Dcel object at 0x7fa9df9660a0>}, '2': {'spec': <Dcel.Dcel object at 0x7fa9df9664f0>, 'file': <Dcel.Dcel object at 0x7fa9df966d30>, 'vfstype': <Dcel.Dcel object at 0x7fa9df966550>, 'mntopts': <Dcel.Dcel object at 0x7fa9df966280>, 'freq': <Dcel.Dcel object at 0x7fa9df966430>, 'passno': <Dcel.Dcel object at 0x7fa9df966520>}, '3': {'spec': <Dcel.Dcel object at 0x7fa9df9b5130>, 'file': <Dcel.Dcel object at 0x7fa9df9b5e80>, 'vfstype': <Dcel.Dcel object at 0x7fa9df9b5fd0>, 'mntopts': <Dcel.Dcel object at 0x7fa9df9b5610>, 'freq': <Dcel.Dcel object at 0x7fa9df9b5850>, 'passno': <Dcel.Dcel object at 0x7fa9df9b5100>}}


In [18]:
cbfs.processRequest(path,updateRequest)

CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}}
CBurnFS::propertyupdate(): path=@/etc/fstabtest.fstab, uprq={'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
CBurnFS::_reinit() called.


{'propertyupdate': 'CBurnFS.propertyupdate():return: improve this response'}

### TEST CBurnFS File Functions

In [19]:
cbfs.getinfo('/boats/skiff.txt').raw

{'basic': {'name': 'skiff.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost', 'file://fs4.localhost']}

In [20]:
cbfs.listdir('/')

['.cosm', 'test.txt', 'fruit', '.cloud', '@', 'boats', '..@', 'numbers']

In [21]:
cbf = Fudge(cbfs)

In [22]:
for each in cbf/'fruit':
    print(each)

red
<i>sakuranbo</i> means <b>cherries</b>



### DEVELOP INIT PROCEDURE

In [4]:
boot = Fudge("file://fs.localhost")
fstab = boot/'.cosm/etc/fstab.fstab/'
cels = []
for ea in fstab:
    if ea/'vfstype' == 'cburnfs':
        urlstr = ea/'spec'
        _scheme = ea/'spec.url/scheme'
        if _scheme == '':
            _scheme = 'file'
        service_class = boot._apath.cosm['services'][str(_scheme)].value
        try:
            cels += [ Dcel(address=str(urlstr), 
                        service_class=service_class) ]
        except:
            pass
root = Dcel(
    service_class=MulticelFS,
    address=cels
)

In [18]:
## Init Procedure

from Fudge import Fudge
from APath import APath
from Dcel import Dcel
from MulticelFS import MulticelFS
from urllib.parse import urlparse

bootpath = 'fs'
boot = Fudge(bootpath)
fstab = boot/'.cosm/etc/fstab'
services = boot._apath.cosm['services']
cels = []
for ea in fstab:
    if fstab/ea/'vfstype' == 'cburnfs':
        urlstr = str(fstab/ea/'spec')
        url = urlparse(urlstr)
        print('url: '+urlstr)
        print('scheme: '+url.scheme)
        print('netloc: '+url.netloc)
        print('path: '+url.path)
        _scheme = url.scheme
        if _scheme == '':
            _scheme = 'file'
        service_class = services[_scheme].value
        print(service_class)
        try:
            d = Dcel(address=str(url.path), 
                     service_class=service_class)
            print(d.listdir('/'))
            cels += [d]
        except:
            raise
        print('')
        
d = MulticelFS(cels)

print(d.listdir('.'))

CreateFailed: unable to create filesystem, 

In [ ]:
from APath import APath
from Dcel import Dcel

services = APath._rootcosm['services']

a = Dcel(address='fs', 
         service_class=services['file']
        )
print(services['file'])

In [4]:
#with CBMetaFS('fs/.cosm/var/cbmeta.zip') as m:
   #m.setinfo('boats',{"cburn":{"hosts":["raygan.com"]}})

#with CBMetaFS('fs/.cosm/var/cbmeta.zip') as m:
    #print(m.getinfo('boats').raw)
    #pass
    
from socket import gethostname



def cbtest(path='/boats/skiff.txt',
           namespaces=None
          ):
    
    print('request: '+path)
    
    hostname = gethostname()

    BlackstrapSvc.initClass()
    BlackstrapSvc.addShare(
        srcaddr = 'fs',
        hostname = hostname,
        sharename = 'fs'
    ).addShare(
        srcaddr = 'fs2',
        hostname = hostname,
        sharename = 'fs2'
    )

    
    cbDemoCosm = {
        'fs': {
            'fs': Dcel('/',BlackstrapSvc(hostname,'fs'),hostname+'/fs'),
            'fs2': Dcel('/',BlackstrapSvc(hostname,'fs2'),hostname+'/fs2'),
            #'cbmeta': Dcel('/',CBMetaFS('fs/.cosm/var/cbmeta.zip')),
        },
        "hosts": [
            "file://iph7rh/fs",
            "file://iph7rh/fs2",
            "meta:.cosm/var/cbmeta.zip"
        ],
        "services": {
            "default": OSFS,
            "file": BlackstrapSvc,
            "meta": CBMetaFS,
            "multi": MultiCelFS
        }
    }
    
    root = CBurnFS(cbDemoCosm)
    #target = root.lookup('abc/def')
    #t2 = target.lookup('ghi')
    #i = t2.getinfo('jkl')

    i = root.getinfo(path,namespaces)
    print(i.raw)
    #print(root.getinfo('boats').raw)
    #print(root.getinfo('colors').raw)
    
    if i.is_dir:
        print(str(root.listdir(path))) 
    else:
        s = root.readbytes(path)
        print(s)

    root.close()
    BlackstrapSvc.closeClass()
    

cbtest("boats")
cbtest("boats/skiff.txt")